# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [11]:
# import libs
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pylab

In [7]:
X, y = datasets.load_boston(return_X_y=True)
print("X.shape", X.shape)

(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.25, random_state=0, shuffle=False)
print("X_train.shape", X_train.shape)
print("X_test.shape", X_test.shape)
print("y_train.shape", y_train.shape)
print("y_test.shape", y_test.shape)

X.shape (506, 13)
X_train.shape (379, 13)
X_test.shape (127, 13)
y_train.shape (379,)
y_test.shape (127,)


In [8]:
def write_answer(index, val):
    with open(f"ans_2_{str(index)}.txt", "w") as fout:
        fout.write(str(val))

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

$L(y, z) = (y - z)^2$

$L'_z(y, z) = 2(z - y)$

$(z - y)$

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [9]:
def gbm_predict(X, base_algorithms_list, coefficients_list):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [ ]:
base_algorithms_list_1 = []
coefficients_list_1 = []
train_scores_1 = []
test_scores_1 = []


def make_regresssor_1(X, y, coef):
    regr = DecisionTreeRegressor(max_depth=5, random_state=42)
    base_algorithms_list_1.append(regr.fit(X, y))
    coefficients_list_1.append(coef)
    
make_regresssor_1(X_train, y_train, 0.9)    
    
for i in range(1, 50):
    make_regresssor_1(X_train, y_train - gbm_predict(X_train, base_algorithms_list_1, coefficients_list_1), 0.9)
    train_scores_1.append(metrics.mean_squared_error(y_train, gbm_predict(X_train, base_algorithms_list_1, coefficients_list_1)))
    test_scores_1.append(metrics.mean_squared_error(y_test, gbm_predict(X_test, base_algorithms_list_1, coefficients_list_1)))

    
pylab.plot(np.arange(1, 50, 1), train_scores_1)   
pylab.plot(np.arange(1, 50, 1), test_scores_1)  
pylab.xlabel('number of trees')
pylab.ylabel('MSE')

In [ ]:
ans_2 = metrics.mean_squared_error(y_test, gbm_predict(X_test, base_algorithms_list_1, coefficients_list_1))**0.5
print("RMSE", ans_2)

write_answer(2, ans_2)

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [ ]:
base_algorithms_list_2 = []
coefficients_list_2 = []
train_scores_2 = []
test_scores_2 = []


def make_regresssor_2(X, y, coef):
    regr = DecisionTreeRegressor(max_depth=5, random_state=42)
    base_algorithms_list_2.append(regr.fit(X, y))
    coefficients_list_2.append(coef)
    
make_regresssor_2(X_train, y_train, 0.9)    
    
for i in range(1, 50):
    make_regresssor_2(X_train, y_train - gbm_predict(X_train, base_algorithms_list_2, coefficients_list_2), 0.9 / (1.0 + i))
    train_scores_2.append(metrics.mean_squared_error(y_train, gbm_predict(X_train, base_algorithms_list_2, coefficients_list_2)))
    test_scores_2.append(metrics.mean_squared_error(y_test, gbm_predict(X_test, base_algorithms_list_2, coefficients_list_2)))

    
pylab.plot(np.arange(1, 50, 1), train_scores_2)   
pylab.plot(np.arange(1, 50, 1), test_scores_2)  
pylab.xlabel('number of trees')
pylab.ylabel('MSE')

In [ ]:
ans_3 = metrics.mean_squared_error(y_test, gbm_predict(X_test, base_algorithms_list_2, coefficients_list_2))**0.5
print("RMSE", ans_3)

write_answer(3, ans_3)

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

     2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

In [ ]:
def make_gbr_mse_est(n_estimators):
    clf = GradientBoostingRegressor(n_estimators=n_estimators)
    clf.fit(X_train, y_train)
    return metrics.mean_squared_error(y_test, clf.predict(X_test)), metrics.mean_squared_error(y_train, clf.predict(X_train))

def make_gbr_mse_d(max_depth):
    clf = GradientBoostingRegressor(max_depth=max_depth)
    clf.fit(X_train, y_train)
    return metrics.mean_squared_error(y_test, clf.predict(X_test)), metrics.mean_squared_error(y_train, clf.predict(X_train))

x_est = np.arange(5, 200, 5)
y_est = [make_gbr_mse_est(x) for x in x_est]
x_d = np.arange(1, 100, 1)
y_d = [make_gbr_mse_d(x) for x in x_d]

pylab.plot(x_est, [y_test for y_test, y_train in y_est], label='n_estimators test')   
pylab.plot(x_est, [y_train for y_est_test, y_train in y_est], label='n_estimators train') 
pylab.plot(x_d, [y_test for y_test, y_train in y_d],  label='max_depth test')
pylab.plot(x_d, [y_train for y_test, y_train in y_d], label='max_depth train')
pylab.legend()

In [ ]:
write_answer(4, "2 3")

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [ ]:
clf = LinearRegression()
clf.fit(X_train, y_train)

ans_5 = metrics.mean_squared_error(y_test, clf.predict(X_test))** 0.5

print("RMSE", ans_5)

write_answer(5, ans_5)